## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-02-21-18 +0000,nypost,Manhunt underway for violent Texas inmate ‘mis...,https://nypost.com/2025/08/19/us-news/manhunt-...
1,2025-08-20-02-21-10 +0000,nyt,Trump’s White House Joins TikTok,https://www.nytimes.com/2025/08/19/us/politics...
2,2025-08-20-02-15-52 +0000,startribune,"Lynx fall 85-75 to Liberty in New York, failin...",https://www.startribune.com/minnesota-lynx-new...
3,2025-08-20-02-09-58 +0000,bbc,"US in talks over 10% Intel stake, White House ...",https://www.bbc.com/news/articles/c620gg77epxo...
4,2025-08-20-02-02-54 +0000,bbc,Dozens of Afghan deportees from Iran killed in...,https://www.bbc.com/news/articles/c336jj5xkvko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,53
462,ukraine,28
67,deal,16
167,zelensky,16
18,house,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
309,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,139
57,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,139
231,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,134
116,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,126
123,2025-08-19-20-23-41 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
309,139,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...
78,74,2025-08-19-21-56-00 +0000,wsj,"Flight Attendants Win Boarding Pay, Four-Year ...",https://www.wsj.com/business/airlines/air-cana...
304,61,2025-08-19-04-24-10 +0000,nypost,White House mocks ‘unserious’ NJ Dem Rep. LaMo...,https://nypost.com/2025/08/19/us-news/white-ho...
52,56,2025-08-19-23-01-18 +0000,nypost,Curtis Sliwa’s quality-of-life crackdown makes...,https://nypost.com/2025/08/19/opinion/i-agree-...
53,43,2025-08-19-23-01-07 +0000,nyt,The Ukrainian children abducted by Russia have...,https://www.nytimes.com/live/2025/08/19/us/tru...
216,37,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
194,36,2025-08-19-16-34-37 +0000,nypost,Memoir from Oct. 7 hostage Eli Sharabi will re...,https://nypost.com/2025/08/19/world-news/memoi...
0,34,2025-08-20-02-21-18 +0000,nypost,Manhunt underway for violent Texas inmate ‘mis...,https://nypost.com/2025/08/19/us-news/manhunt-...
268,32,2025-08-19-10-00-00 +0000,latimes,A deadly crash and Musk's exaggerations: Insid...,https://www.latimes.com/business/story/2025-08...
28,32,2025-08-20-00-40-22 +0000,nypost,Husband of woman found dead in national forest...,https://nypost.com/2025/08/19/us-news/husband-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
